In [ ]:
import os
import glob
import scipy
import pandas as pd
import seaborn as sns
from scipy import stats
import warnings
from matplotlib.colors import ListedColormap
import random
%matplotlib
%pylab inline
sns.set(font_scale=1.6)
sns.set_style('whitegrid')

aggregated_order= ['<= -0.2','(-0.2--0.15]','(-0.15--0.1]','(-0.1--0.05]',\
                   '(-0.05-0.0]','(0.0-0.05]','(0.05-0.1]','(0.1-0.15]','(0.15-0.2]','> 0.2']

sp_mp_bins=['[0.0-0.1]', '(0.1-0.2]', '(0.2-0.3]', '(0.3-0.4]', '(0.4-0.5]',\
            '(0.5-0.6]', '(0.6-0.7]', '(0.7-0.8]', '(0.8-0.9]', '(0.9-1.0]']

sns.__version__

import matplotlib as mpl
mpl.rcParams
COLOR = '#003985'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

print('Currently under development')

In [ ]:
company=''
T8c=f''

# PLOT COMPANY METRICS

In [ ]:
aggregated_delta_tasks=pd.DataFrame()
for idx,f in enumerate(sorted(glob.glob(f'{company}/cls/deltas_global_performances.csv'))):
    company_df=pd.read_csv(f)
    print(f)
    company_df['Company']=f'{idx+1}'
    aggregated_delta_tasks=pd.concat((aggregated_delta_tasks,company_df),sort=False)
n=str(len(aggregated_delta_tasks.Company.unique()))
g=sns.catplot(data=aggregated_delta_tasks[[col for col in aggregated_delta_tasks.columns if col not in ['Company','kappa','positive_rate']]].melt(),y='value',x='variable',kind='bar',color='#2081F3',aspect=2,height=6)
for i, ax in enumerate(g.fig.axes):   ## getting all axes of the fig object
     ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
g.set_ylabels(f"∆MPvs.SP\n(Cross company deviation)")
g.set_xlabels(f'Performance Metric\n(N={n})')
plt.axhline(0.00)

In [ ]:
#get aggregate deltas across pharma & anonymise
aggregated_delta_tasks=pd.DataFrame()
for f in sorted(glob.glob(f'{company}/cls/deltas_per-assay_performances.csv')):
    company_df=pd.read_csv(f)
    print(f)
    company_df['Company']=f.split('/')[0]
    aggregated_delta_tasks=pd.concat((aggregated_delta_tasks,company_df),sort=False)
aggregated_delta_tasks=aggregated_delta_tasks[[col for col in aggregated_delta_tasks.columns if col not in ['Company','kappa','positive_rate']]]
n=str(aggregated_delta_tasks.groupby('assay_type').count().iloc[0][0])
g=sns.catplot(data=aggregated_delta_tasks.melt(id_vars=['assay_type']),y='value',x='variable',kind='bar',col='assay_type',col_wrap=2,aspect=2,height=6)
for i, ax in enumerate(g.fig.axes):   ## getting all axes of the fig object
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
g.set_titles(row_template = '           {row_name}           ', col_template = '{col_name} (N='+n+')')
g.set_ylabels(f"∆MPvs.SP")
g.set_xlabels('Performance\nMetric')

# PLOT PREDICTION RATES

In [ ]:
#get aggregate deltas across pharma & anonymise
aggregated_delta_tasks=pd.DataFrame()
for f in sorted(glob.glob(f'{company}/cls/deltas_per-assay_performances.csv')):
    company_df=pd.read_csv(f)
    print(f)
    company_df['Company']=f.split('/')[0]
    aggregated_delta_tasks=pd.concat((aggregated_delta_tasks,company_df),sort=False)
aggregated_delta_tasks=aggregated_delta_tasks[[col for col in aggregated_delta_tasks.columns if col in ['assay_type','tpr','fnr','fpr','tnr']]]
n=str(aggregated_delta_tasks.groupby('assay_type').count().iloc[0][0])
g=sns.catplot(data=aggregated_delta_tasks.melt(id_vars=['assay_type']),y='value',x='variable',kind='bar',col='assay_type',col_wrap=2,aspect=2,height=6)
for i, ax in enumerate(g.fig.axes):   ## getting all axes of the fig object
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
g.set_titles(row_template = '           {row_name}           ', col_template = '{col_name} (N='+n+')')
g.set_ylabels(f"∆MPvs.SP")
g.set_xlabels('Performance\nMetric')

# CONFUSION MATRIX PER ASSAY TYPE

In [ ]:
cm=aggregated_delta_tasks.melt(id_vars=['assay_type']).query('variable in ["tpr","fnr","tnr","fpr"]')
mmin=aggregated_delta_tasks.melt(id_vars=['assay_type']).query('variable in ["tpr","fnr","tnr","fpr"]')['value'].min()
mmax=aggregated_delta_tasks.melt(id_vars=['assay_type']).query('variable in ["tpr","fnr","tnr","fpr"]')['value'].max()
for at,atdf in aggregated_delta_tasks.groupby('assay_type'):
    ndf=pd.concat((atdf[['tpr','fpr']].T.reset_index(drop=True).T,atdf[['fnr','tnr']].T.reset_index(drop=True).T),axis=0)
    ndf.columns=['A','I']
    ndf.index=['A','I']
    ax=sns.heatmap(ndf,annot=True,cmap='RdYlGn',center=0,vmin=mmin,vmax=mmax)
    ax.set_title(at)
    ax.set_ylabel('Predicted')
    ax.set_xlabel('Actual')
    plt.show()

In [ ]:
company_df=pd.read_csv(f'{company}/cls/deltas_per-task_performances.csv')
company_t8c=pd.read_csv(T8c)
company_df=company_df.merge(company_t8c,left_on='classification_task_id',right_on='classification_task_id',how='left')
company_df['size']=company_df['num_total_inactives']+company_df['num_total_actives']

# EFFECT OF PRAUC CALIBRATION (WP1 DISCUSSION)

In [ ]:
company_df=pd.read_csv(f'{company}/cls/deltas_per-task_performances.csv')
company_t8c=pd.read_csv(T8c)
company_df=company_df.merge(company_t8c,left_on='classification_task_id',right_on='classification_task_id',how='left')
company_df['size']=company_df['num_total_inactives']+company_df['num_total_actives']

from scipy import stats
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

g=sns.jointplot(data=company_df,x='auc_pr',y='calibrated_auc_pr')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle(f"r2={round(r2(company_df['auc_pr'],company_df['calibrated_auc_pr']),3)}")

In [ ]:
g.savefig('wp1_toshare1.png')

In [ ]:
company_df['pc_size']=company_df['size'].rank(pct=True)*100
company_df['abs_difference_aucpr']=(company_df['calibrated_auc_pr']-company_df['auc_pr']).abs()
g=sns.jointplot(data=company_df,x='pc_size',y='abs_difference_aucpr')
#g.ax_joint.set_xscale('log')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle(f"r2={round(r2(company_df['abs_difference_aucpr'],company_df['size']),3)}")

In [ ]:
g.savefig('wp1_toshare2.png')

In [ ]:
def cut(x, bins, lower_infinite=True, upper_infinite=True, **kwargs):        
    num_labels      = len(bins) - 1
    include_lowest  = kwargs.get("include_lowest", False)
    right           = kwargs.get("right", True)
    bins_final = bins.copy()
    if upper_infinite:
        bins_final.insert(len(bins),float("inf"))
        num_labels += 1
    if lower_infinite:
        bins_final.insert(0,float("-inf"))
        num_labels += 1
    symbol_lower  = "<=" if include_lowest and right else "<"
    left_bracket  = "(" if right else "["
    right_bracket = "]" if right else ")"
    symbol_upper  = ">" if right else ">="
    labels=[]
    
    def make_label(i, lb=left_bracket, rb=right_bracket):
        return "{0}{1}-{2}{3}".format(lb, bins_final[i], bins_final[i+1], rb)
        
    for i in range(0,num_labels):
        new_label = None
        if i == 0:
            if lower_infinite:
                new_label = "{0} {1}".format(symbol_lower, bins_final[i+1])
            elif include_lowest:
                new_label = make_label(i, lb="[")
            else:
                new_label = make_label(i)
        elif upper_infinite and i == (num_labels - 1):
            new_label = "{0} {1}".format(symbol_upper, bins_final[i])
        else:
            new_label = make_label(i)
        labels.append(new_label)
    return pd.cut(x, bins_final, labels=labels, **kwargs)

mp_company_df=pd.read_csv(f'{company}/cls/MP/pred_per-task_performances.csv')
sp_company_df=pd.read_csv(f'{company}/cls/SP/pred_per-task_performances.csv')
company_df=mp_company_df[['classification_task_id','auc_pr','calibrated_auc_pr']].merge(sp_company_df[['classification_task_id','auc_pr','calibrated_auc_pr']],left_on='classification_task_id',right_on='classification_task_id',how='outer',suffixes=('_MP', '_SP'))
company_t8c=pd.read_csv(T8c)
company_df['size']=mp_company_df['num_total_actives']
company_df['taskbin'] = cut(company_df['size'].astype('float32'), \
[75,100,500,1000],include_lowest=True,right=True)

toplot=company_df.melt(id_vars=['classification_task_id','taskbin'],value_vars=['auc_pr_MP','calibrated_auc_pr_MP','auc_pr_SP','calibrated_auc_pr_SP'])
order=sorted(toplot['variable'].unique())

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(data=toplot,
            y='variable',
            x='value',
            orient='h',
            showfliers=False,
            order=order, 
            ax=ax)

plt.subplots_adjust(left=0.35, bottom=0.15, right=1, top=0.9)
plt.savefig('wp1_toshare3.png')

# PREDICTION RATE CORRELATION

In [ ]:
for at,atdf in company_df.groupby('assay_type_x'):
    ax=sns.heatmap(atdf[['auc_pr','tnr','tpr','fnr','fpr']].corr(),mask=np.triu(atdf[['auc_pr','tnr','tpr','fnr','fpr']].corr()),annot=True,cmap='RdYlGn',center=0,vmin=-1,vmax=1)
    ax.set_title(at)
    plt.show()

# DISTRIBUTION OF RATES ACROSS TASKS

In [ ]:
plt.figure(figsize=(10,10))
ax=sns.boxplot(data=company_df.melt(id_vars='assay_type_x',value_vars=['tpr','tnr','fpr','fnr']), \
           x='variable',y='value',hue='assay_type_x')
ax.set_ylabel('Delta (cross task distribution)')
ax.set_xlabel('Prediction rate')
plt.axhline(0,linewidth=4,color='grey',linestyle='--')

In [ ]:
hdata=company_df[['assay_type_x','tpr','fnr','tpr','fpr','auc_pr']].set_index('assay_type_x')
lut = dict(zip(hdata.index.unique(), "rbg"))
row_colors = hdata.index.map(lut)

# BI CLUSTERED HEATMAP OF PREDICTION RATE EFFECTS

In [ ]:
ax=sns.clustermap(company_df[['assay_type_x','tpr','fnr','tnr','fpr','auc_pr']].set_index('assay_type_x').T, \
                  col_colors=row_colors,cmap="vlag",figsize=(15, 5),method='complete',metric="euclidean", \
                  colors_ratio=.3, dendrogram_ratio=(.01, .2), xticklabels=False,cbar_pos=(.97, 0.23,0.05, 0.18))

from matplotlib.patches import Patch

handles = [Patch(facecolor=lut[name]) for name in lut]
plt.legend(handles, lut, title='assay_type',
           bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure, loc='upper center')

In [ ]:
for tdf,tdf2 in company_df.groupby('assay_type_x'):
    ax=sns.clustermap(tdf2[['assay_type_x','tpr','fnr','tnr','fpr','auc_pr']].set_index('assay_type_x').T, \
                      cmap="vlag",figsize=(15, 5),method='complete',metric="euclidean", \
                      dendrogram_ratio=(.01, .2), xticklabels=False,cbar_pos=(.97, 0.23,0.05, 0.18)).fig.suptitle(tdf)

# EFFECT OF TOTAL TASK SIZE ON PERFORMANCE

In [ ]:
sns.relplot(data=company_df,x='auc_pr',y='size',height=8,alpha=.5)

In [ ]:
sns.relplot(data=company_df,x='auc_pr',y='size',height=10,col='assay_type_x',col_wrap=2,alpha=.5)

In [ ]:
sns.relplot(data=company_df,y='num_total_actives',x='auc_pr',height=8,alpha=.5)

# SIGNIFICANT IMPROVED OR DETERIORATED TASK

In [ ]:
toplot=company_df.query('(mp_significant==1) | (sp_significant==1)').groupby(['assay_type_x','mp_significant']).count().reset_index()
toplot['Significance']=toplot['mp_significant'].astype(str).str.replace('0','SP').str.replace('1','MP')
sns.catplot(data=toplot,col='assay_type_x',x='Significance',y='classification_task_id',kind='bar').set_axis_labels("","no. tasks significant")

In [ ]:
dfdict=company_df.groupby('assay_type_x').count()['classification_task_id'].to_dict()
def compute_percentage(x):
    pct = (x['classification_task_id']/dfdict[x.index[0][0]] * 100)
    return round(pct, 2)
ndf=company_df.query('(mp_significant==1) | (sp_significant==1)').groupby(['assay_type_x','mp_significant']).count()
ndf2=pd.DataFrame(ndf.groupby(['assay_type_x','mp_significant']).apply(compute_percentage))
ndf2.index=ndf2.index.droplevel([0,1])
ndf2=ndf2.reset_index()
ndf2['Significance']=ndf2['mp_significant'].astype(str).str.replace('0','SP').str.replace('1','MP')
ax=sns.catplot(data=ndf2,col='assay_type_x',x='Significance',y='classification_task_id',kind='bar').set_axis_labels("","% tasks significant")

In [ ]:
company_df['Significant']=company_df['mp_significant'].astype(str) + company_df['sp_significant'].astype(str)
company_df['Significant']=company_df['Significant'].str.replace('00','').str.replace('10','MP').str.replace('01','SP')

In [ ]:
sns.relplot(data=company_df,x='auc_pr',y='roc_auc_score',col='assay_type_x',hue='Significant',alpha=0.75,height=10,s=130,marker='o')

In [ ]:
hdata=company_df[['assay_type_x','Significant']].set_index('Significant')
lut = dict(zip(hdata.index.unique(), "wky"))
row_colors = hdata.index.map(lut)

hdata=company_df[['assay_type_x','Significant']].set_index('assay_type_x')
lut2 = dict(zip(hdata.index.unique(), "rbg"))
row_colors2 = hdata.index.map(lut2)

ax=sns.clustermap(company_df[company_df.auc_pr.abs()>0.05][['assay_type_x','auc_pr','roc_auc_score']].set_index('assay_type_x').T, \
                  col_colors=[row_colors,row_colors2],cmap="vlag",figsize=(15, 5),method='complete',metric="euclidean", \
                  colors_ratio=.15, dendrogram_ratio=(.01, .2), xticklabels=False,cbar_pos=(.97, 0.23,0.05, 0.18))

from matplotlib.patches import Patch

handles = [Patch(facecolor=lut[name]) for name in lut]
plt.legend(handles, lut, title='Significance',
           bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure, loc='upper center')

handles = [Patch(facecolor=lut2[name]) for name in lut2]
plt.legend(handles, lut2, title='Significance',
           bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure, loc='lower center')

In [ ]:
sns.boxplot(data=company_df,y='auc_pr',x='assay_type_x',hue='direction')